In [1]:
!pip install fastapi uvicorn python-socketio pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 5.9 MB/s eta 0:00:00


In [3]:
# server.py
import time
from typing import Dict
from fastapi import FastAPI
from fastapi.staticfiles import StaticFiles
import socketio

# --- Socket.IO server ---
sio = socketio.AsyncServer(
    async_mode="asgi",
    cors_allowed_origins="*",
    ping_timeout=30,
    ping_interval=25,
)
app = FastAPI()
sio_app = socketio.ASGIApp(sio, other_asgi_app=app)

# Serve static files (index.html will be created later)
app.mount("/", StaticFiles(directory="public", html=True), name="static")

online_users: Dict[str, str] = {}

@sio.event
async def connect(sid, environ, auth):
    print("connected:", sid)

@sio.event
async def join(sid, data):
    room = data.get("room", "global")
    username = data.get("username", f"user-{sid[:5]}")
    online_users[sid] = username
    await sio.enter_room(sid, room)
    await sio.emit("system", {"text": f"{username} joined #{room}", "room": room, "ts": time.time()}, room=room)
    await sio.emit("presence", {"online": list(set(online_users.values()))}, room=room)

@sio.event
async def typing(sid, data):
    room = data.get("room", "global")
    await sio.emit("typing", data, room=room, skip_sid=sid)

@sio.event
async def message(sid, data):
    room = data.get("room", "global")
    payload = {
        "room": room,
        "username": data.get("username", online_users.get(sid, "anon")),
        "text": data.get("text", ""),
        "ts": time.time(),
    }
    await sio.emit("message", payload, room=room)

@sio.event
async def leave(sid, data):
    room = data.get("room", "global")
    username = online_users.get(sid, f"user-{sid[:5]}")
    await sio.leave_room(sid, room)
    await sio.emit("system", {"text": f"{username} left #{room}", "room": room, "ts": time.time()}, room=room)

@sio.event
async def disconnect(sid):
    username = online_users.pop(sid, None)
    print("disconnected:", sid)
    await sio.emit("presence", {"online": list(set(online_users.values()))})


In [20]:
!pip install gradio

import gradio as gr
import time

# Shared message history for all users
messages = []

def add_message(user, text):
    """Add a new message and return the updated chat log."""
    timestamp = time.strftime("%H:%M:%S")
    messages.append((f"{user} ({timestamp})", text))
    return messages

def chat(user, message, history):
    """Handle sending a message."""
    if user.strip() == "":
        user = "Anonymous"
    return add_message(user, message)

# Build the UI
with gr.Blocks() as demo:
    gr.Markdown("## 🗨️ Realtime Python Chat (Gradio + Colab)")

    with gr.Row():
        username = gr.Textbox(label="Your Name", placeholder="Enter username")

    chatbot = gr.Chatbot(label="Chat Room")
    msg = gr.Textbox(placeholder="Type your message... (Press Enter to send)")

    # When user sends a message
    msg.submit(chat, [username, msg, chatbot], chatbot)

demo.launch(share=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.5/324.5 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 124.7 MB/s eta 0:00:00


/tmp/ipython-input-777244288.py:28: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat Room")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://63ad5dc11461d79494.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
